In [1]:
import streamlit as st
import sys
sys.path.append('src')

# Configure page
st.set_page_config(
    page_title="Wind Power Forecasting",
    page_icon="🌊",
    layout="wide"
)

# Sidebar navigation
st.sidebar.title("Navigation")
page = st.sidebar.radio(
    "Go to",
    ["Home", "Data Exploration", "Correlation Analysis", "Model Training", "Results"]
)

# Home Page
if page == "Home":
    st.title("🌊 Wind Power Forecasting with Wave Data")
    st.markdown("""
    ## Project Overview
    This project investigates wave-enhanced wind power forecasting using data from 
    the Irish Marine Data Buoy Network.
    
    ### Features:
    - Data sourced from Marine Institute ERDDAP server
    - Correlation analysis between wind and wave parameters
    - Time-lagged cross-correlation analysis
    - Baseline vs Wave-Enhanced model comparison
    
    **Select a page from the sidebar to explore the analysis.**
    """)

# Data Exploration Page
elif page == "Data Exploration":
    st.title("📊 Data Exploration")
    
    from irish_buoy_data import IrishBuoyData
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Sidebar controls
    station = st.sidebar.selectbox("Select Buoy Station", ["M6", "M5", "M4", "M3", "M2", "M1"])
    days = st.sidebar.slider("Days of Data", 7, 365, 30)
    
    # Fetch data
    with st.spinner('Fetching data...'):
        buoy = IrishBuoyData(station_id=station)
        data = buoy.fetch_data(days_back=days)
    
    st.success(f"✅ Loaded {len(data)} records from {station}")
    
    # Display data info
    col1, col2, col3 = st.columns(3)
    col1.metric("Total Records", len(data))
    col2.metric("Date Range", f"{(data.index.max() - data.index.min()).days} days")
    col3.metric("Missing Values", data.isnull().sum().sum())
    
    # Show data sample
    st.subheader("Data Sample")
    st.dataframe(data.head(10))
    
    # Plot time series
    st.subheader("Time Series Visualization")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    axes[0, 0].plot(data.index, data['WindSpeed (knots)'], linewidth=1)
    axes[0, 0].set_title('Wind Speed')
    axes[0, 0].set_ylabel('knots')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    axes[0, 1].plot(data.index, data['AirTemperature (degrees_C)'], color='coral', linewidth=1)
    axes[0, 1].set_title('Air Temperature')
    axes[0, 1].set_ylabel('°C')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    axes[1, 0].plot(data.index, data['WaveHeight (meters)'], color='navy', linewidth=1)
    axes[1, 0].set_title('Wave Height')
    axes[1, 0].set_ylabel('meters')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    axes[1, 1].plot(data.index, data['Hmax (meters)'], color='red', linewidth=1)
    axes[1, 1].set_title('Max Wave Height')
    axes[1, 1].set_ylabel('meters')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    st.pyplot(fig)

# Correlation Analysis Page
elif page == "Correlation Analysis":
    st.title("🔗 Correlation Analysis")
    
    from irish_buoy_data import IrishBuoyData
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Fetch data
    station = st.sidebar.selectbox("Select Buoy Station", ["M6", "M5", "M4"])
    days = st.sidebar.slider("Days of Data", 30, 365, 180)
    
    with st.spinner('Analyzing correlations...'):
        buoy = IrishBuoyData(station_id=station)
        data = buoy.fetch_data(days_back=days)
    
    # Calculate correlation matrix
    numeric_data = data.select_dtypes(include=[np.number])
    corr_matrix = numeric_data.corr()
    
    # Display key correlation
    wave_wind_corr = corr_matrix.loc['WaveHeight (meters)', 'WindSpeed (knots)']
    
    col1, col2, col3 = st.columns(3)
    col1.metric("Wind-Wave Correlation", f"{wave_wind_corr:.3f}")
    col2.metric("Paper Reported", "0.72")
    col3.metric("Difference", f"{abs(wave_wind_corr - 0.72):.3f}")
    
    # Correlation heatmap
    st.subheader("Correlation Matrix")
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, ax=ax)
    st.pyplot(fig)
    
    # Scatter plots
    st.subheader("Relationship Visualizations")
    col1, col2 = st.columns(2)
    
    with col1:
        fig, ax = plt.subplots()
        ax.scatter(data['WindSpeed (knots)'], data['WaveHeight (meters)'], alpha=0.5, s=10)
        ax.set_xlabel('Wind Speed (knots)')
        ax.set_ylabel('Wave Height (m)')
        ax.set_title(f'r = {wave_wind_corr:.3f}')
        st.pyplot(fig)
    
    with col2:
        fig, ax = plt.subplots()
        ax.scatter(data['WindSpeed (knots)'], data['Hmax (meters)'], alpha=0.5, s=10, color='red')
        ax.set_xlabel('Wind Speed (knots)')
        ax.set_ylabel('Max Wave Height (m)')
        corr_hmax = corr_matrix.loc['Hmax (meters)', 'WindSpeed (knots)']
        ax.set_title(f'r = {corr_hmax:.3f}')
        st.pyplot(fig)

# Model Training Page
elif page == "Model Training":
    st.title("🤖 Model Training")
    st.info("Model training interface - Coming Soon!")
    
    st.markdown("""
    ### Planned Features:
    - Train/Test split configuration
    - Model selection (LSTM, GRU, etc.)
    - Hyperparameter tuning
    - Training progress visualization
    """)

# Results Page
elif page == "Results":
    st.title("📈 Results & Comparison")
    st.info("Results dashboard - Coming Soon!")
    
    st.markdown("""
    ### Planned Comparisons:
    - Baseline (Met-only) vs Wave-Enhanced models
    - Forecast accuracy metrics (RMSE, MAE, R²)
    - Error analysis
    - Feature importance
    """)

2025-12-25 03:06:20.218 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-25 03:06:20.220 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-25 03:06:20.345 
  command:

    streamlit run /opt/anaconda3/envs/streamlit_env/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-12-25 03:06:20.345 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-25 03:06:20.346 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-25 03:06:20.347 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-25 03:06:20.347 Thread 'MainThread': missing ScriptRunContext! This warning can 